<a href="https://colab.research.google.com/github/rberbenkova/lab-sql-generation-with-transformer-api/blob/main/lab-sql-generation-with-transformer-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Generation with Transformer API

In [42]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [44]:
torch.cuda.is_available()

True

In [45]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [46]:
print(available_memory)

15828320256


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [47]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [8]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [9]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [10]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [11]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

---3 PROMT VERSIONS (baseline, schema-anchored strict, and
self-checking)

**Example questions to try**
**Revenue by product (top 5)**: Which 5 products generated the highest total revenue overall?

**Gross margin by salesperson:** Which salesperson had the highest total margin (revenue - cost) last year?

**Top customers by spend:** List top 10 customers by total revenue in 2024.

**Impossible** (should return I do not know): List the email addresses of customers who bought frankfurts. (No emails in schema.)

# Version 1 — Baseline (polite, minimal rules)

In [48]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If the question cannot be answered with the schema, return exactly: I do not know
- Revenue = products.price * sales.quantity
- Cost = product_suppliers.supply_price * sales.quantity

### Database Schema
This query will run on a database whose schema is represented in this string:

CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);
CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  address VARCHAR(100)
);
CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);
CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);
CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- Joins:
-- sales.product_id = products.product_id
-- sales.customer_id = customers.customer_id
-- sales.salesperson_id = salespeople.salesperson_id
-- product_suppliers.product_id = products.product_id

### Answer
Return only a single SQL statement. No commentary.
"""


In [50]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [ ]:
question = "Which 5 products generated the highest total revenue overall?"
generated_sql = generate_query(question)

In [15]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION]Which 5 products generated the highest total revenue overall?[/QUESTION] ### Instructions - If the question cannot be answered with the schema,
                                                                                                                                                                                          return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity ### Database Schema This query will run on a database whose schema is represented in this string:
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name VARCHAR(50),
              

In [16]:
question = "Which salesperson had the highest total margin (revenue - cost) last year?"
generated_sql = generate_query(question)

In [17]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION]Which salesperson had the highest total margin (revenue - cost) last year?[/QUESTION] ### Instructions - If the question cannot be answered with the schema,
                                                                                                                                                                                                       return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity ### Database Schema This query will run on a database whose schema is represented in this string:
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name V

In [18]:
question = "List top 10 customers by total revenue in 2024."
generated_sql = generate_query(question)

In [19]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION]List top 10 customers by total revenue in 2024.[/QUESTION] ### Instructions - If the question cannot be answered with the schema,
                                                                                                                                                                            return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity ### Database Schema This query will run on a database whose schema is represented in this string:
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name VARCHAR(50),
                                          

In [20]:
question = " List the email addresses of customers who bought frankfurts. (No emails in schema.)"
generated_sql = generate_query(question)

In [21]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION] List the email addresses of customers who bought frankfurts. (No emails in schema.)[/QUESTION] ### Instructions - If the question cannot be answered with the schema,
                                                                                                                                                                                                                 return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity ### Database Schema This query will run on a database whose schema is represented in this string:
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                              

# Version 2 — Strict schema-anchored (format & safety guards)

In [22]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]
You are an expert SQL generator.

RULES
1) Output ONLY SQL (no prose, no comments), or exactly: I do not know
2) Use ONLY columns/tables from the provided schema
3) Use table aliases and explicit JOINs
4) If a required attribute isn't in the schema, return: I do not know
5) Revenue = products.price * sales.quantity; Cost = product_suppliers.supply_price * sales.quantity
6) Prefer CTEs for multi-step logic

SCHEMA
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);
CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  address VARCHAR(100)
);
CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);
CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);
CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- Joins:
-- sales.product_id = products.product_id
-- sales.customer_id = customers.customer_id
-- sales.salesperson_id = salespeople.salesperson_id
-- product_suppliers.product_id = products.product_id

QUESTION: {question}

RETURN: a single valid SQL statement compatible with standard Postgres.
"""

In [23]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [24]:
question = "Which 5 products generated the highest total revenue overall?"
generated_sql = generate_query(question)

In [25]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION]Which 5 products generated the highest total revenue overall?[/QUESTION] You are an expert SQL generator. RULES 1) Output ONLY SQL (no prose,
                                                                                                                                                                                         no comments),
or exactly: I do not know 2) Use ONLY columns/tables
from the provided schema 3) Use table aliases
and explicit JOINs 4) If a required attribute isn't in the schema,
                                                           return: I do not know 5) Revenue = products.price * sales.quantity;

Cost = product_suppliers.supply_price * sales.quantity 6) Prefer CTEs
for multi-step logic SCHEMA
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                 

In [26]:
question = "Which salesperson had the highest total margin (revenue - cost) last year?"
generated_sql = generate_query(question)

In [27]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION]Which salesperson had the highest total margin (revenue - cost) last year?[/QUESTION] You are an expert SQL generator. RULES 1) Output ONLY SQL (no prose,
                                                                                                                                                                                                      no comments),
or exactly: I do not know 2) Use ONLY columns/tables
from the provided schema 3) Use table aliases
and explicit JOINs 4) If a required attribute isn't in the schema, return: I do not know
5) Revenue = products.price * sales.quantity; Cost = product_suppliers.supply_price * sales.quantity
6) Prefer CTEs for multi-step logic

SCHEMA
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);
CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  address VARCHAR(100)
);
CREATE TABLE salespeo

In [28]:
question = "List top 10 customers by total revenue in 2024."
generated_sql = generate_query(question)

In [29]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION]List top 10 customers by total revenue in 2024.[/QUESTION] You are an expert SQL generator. RULES 1) Output ONLY SQL (no prose,
                                                                                                                                                                           no comments),
or exactly: I do not know 2) Use ONLY columns/tables
from the provided schema 3) Use table aliases
and explicit JOINs 4) If a required attribute isn't in the schema,
                                                           return: I do not know 5) Revenue = products.price * sales.quantity;

Cost = product_suppliers.supply_price * sales.quantity 6) Prefer CTEs
for multi-step logic SCHEMA
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity

In [30]:
question = " List the email addresses of customers who bought frankfurts. (No emails in schema.)"
generated_sql = generate_query(question)

In [31]:
print(generated_sql)

### Task Generate a SQL query to answer [QUESTION] List the email addresses of customers who bought frankfurts. (No emails in schema.)[/QUESTION] You are an expert SQL generator. RULES 1) Output ONLY SQL (no prose,
                                                                                                                                                                                                                no comments),
or exactly: I do not know 2) Use ONLY columns/tables
from the provided schema 3) Use table aliases
and explicit JOINs 4) If a required attribute isn't in the schema, return: I do not know
5) Revenue = products.price * sales.quantity; Cost = product_suppliers.supply_price * sales.quantity
6) Prefer CTEs for multi-step logic

SCHEMA
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);
CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  address VARCHAR(100)
);
C

# Version 3 — Self-checking (validator-style)

In [ ]:
prompt = """###

ROLE: SQL author + validator.

TASK: Produce a correct SQL query for QUESTION using SCHEMA.

CONSTRAINTS:
- If not answerable from SCHEMA, return exactly: I do not know
- Revenue = products.price * sales.quantity
- Cost = product_suppliers.supply_price * sales.quantity
- Use explicit JOINs, table aliases, and CTEs for clarity
- No references to non-existent columns

QUESTION: {question}

SCHEMA
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);
CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  address VARCHAR(100)
);
CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);
CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);
CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);
-- Joins:
-- sales.product_id = products.product_id
-- sales.customer_id = customers.customer_id
-- sales.salesperson_id = salespeople.salesperson_id
-- product_suppliers.product_id = products.product_id

PLAN (silently): verify each selected column exists; verify each join key exists; verify aggregations match SELECT.
RETURN: Output ONLY the final SQL (no explanations).
"""


In [33]:

import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [40]:
question = "Which 5 products generated the highest total revenue overall?"
generated_sql = generate_query(question)


In [41]:
print(generated_sql)

### ROLE: SQL author + validator. TASK: Produce a correct SQL query
for QUESTION using SCHEMA. CONSTRAINTS: - If not answerable
from SCHEMA,
     return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity - Use explicit JOINs,
                                                                                                                                              table aliases,
and CTEs
for clarity - No references to non-existent columns QUESTION: Which 5 products generated the highest total revenue overall? SCHEMA
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name VARCHAR(50),
                           

In [ ]:
question = "Which salesperson had the highest total margin (revenue - cost) last year?"
generated_sql = generate_query(question)

In [36]:
print(generated_sql)

### ROLE: SQL author + validator. TASK: Produce a correct SQL query
for QUESTION using SCHEMA. CONSTRAINTS: - If not answerable
from SCHEMA,
     return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity - Use explicit JOINs,
                                                                                                                                              table aliases,
and CTEs
for clarity - No references to non-existent columns QUESTION: Which 5 products generated the highest total revenue overall? SCHEMA
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name VARCHAR(50),
                           

In [ ]:
question = "List top 10 customers by total revenue in 2024."
generated_sql = generate_query(question)

In [37]:
print(generated_sql)

### ROLE: SQL author + validator. TASK: Produce a correct SQL query
for QUESTION using SCHEMA. CONSTRAINTS: - If not answerable
from SCHEMA,
     return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity - Use explicit JOINs,
                                                                                                                                              table aliases,
and CTEs
for clarity - No references to non-existent columns QUESTION: Which 5 products generated the highest total revenue overall? SCHEMA
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name VARCHAR(50),
                           

In [38]:
question = " List the email addresses of customers who bought frankfurts. (No emails in schema.)"
generated_sql = generate_query(question)

In [39]:
print(generated_sql)

### ROLE: SQL author + validator. TASK: Produce a correct SQL query
for QUESTION using SCHEMA. CONSTRAINTS: - If not answerable
from SCHEMA,
     return exactly: I do not know - Revenue = products.price * sales.quantity - Cost = product_suppliers.supply_price * sales.quantity - Use explicit JOINs,
                                                                                                                                              table aliases,
and CTEs
for clarity - No references to non-existent columns QUESTION: List the email addresses of customers who bought frankfurts. (No emails in schema.) SCHEMA
CREATE TABLE products (product_id INTEGER PRIMARY KEY,
                                          name VARCHAR(50),
                                               price DECIMAL(10, 2),
                                                     quantity INTEGER);


CREATE TABLE customers (customer_id INTEGER PRIMARY KEY,
                                            name VARCHAR(50),
     



# Example questions to try


**Revenue by product (top 5):**
Which 5 products generated the highest total revenue overall?


**Gross margin by salesperson:**
Which salesperson had the highest total margin (revenue - cost) last year?


**Top customers by spend:**
List top 10 customers by total revenue in 2024.


**Impossible** (should return I do not know):
List the email addresses of customers who bought frankfurts.
(No emails in schema.)

# **One-page report**

**Prompt Engineering for SQL Generation: Findings & Lessons**

**Overview.**
I evaluated three prompt styles to generate SQL from a fixed retail schema: (V1) Baseline, (V2) Strict schema-anchored, and (V3) Self-checking validator. I asked questions about revenue, margin, and rankings, plus an impossible question to test refusals. Revenue was defined as products.price * sales.quantity and cost as product_suppliers.supply_price * sales.quantity.

**Results.**

V1 (Baseline): Often produced correct SQL for straightforward joins (sales→products/customers). However, it sometimes added prose (“Summary: …”) or used implicit joins. On the impossible question (emails), it occasionally attempted to guess a column instead of refusing, yielding hallucinated fields.

V2 (Strict): Reliability improved markedly. Output was usually a single SQL statement with explicit JOINs and clear aliases. It correctly refused with exactly “I do not know” for unavailable attributes (e.g., emails). It followed the margin/revenue definitions consistently.

V3 (Self-checking): Matched V2’s accuracy and further reduced schema mistakes. It almost never referenced non-existent columns and respected the “only SQL” constraint. For multi-step logic (e.g., margin by salesperson and timeframe), V3 used CTEs cleanly. The validator mindset helped avoid aggregation mismatches (GROUP BY vs SELECT).

What didn’t work well / hallucinations.

Lax constraints (V1). Without explicit “ONLY SQL / no prose,” outputs sometimes included commentary or markdown fences.

Missing attributes. When the question requested unschematized data (e.g., emails, discounts), V1 sometimes invented columns. V2/V3 fixed this with strict refusal language.

Ambiguous business terms. If the question asked for “profit” rather than the defined “margin = revenue - cost,” V1 occasionally diverged (e.g., ignoring cost); V2/V3 were better once instructions defined margin precisely.

**Key prompt techniques that helped.**

Schema anchoring & join map reduce table/column hallucinations.

“Only SQL or ‘I do not know’” plus no-prose formatting keeps outputs usable.

Explicit definitions (revenue, cost, margin) eliminate semantic drift.

CTE encouragement creates clearer, safer queries for multi-step calculations.

Self-validation (V3) cuts errors: “verify each column exists; check join keys; ensure GROUP BY matches.”

**What I learned.**

Strong guardrails (format, refusal rules) dramatically improve reliability.

Giving the model precise business formulas prevents subtle mistakes.

Self-checking language (even without showing the reasoning) reduces column/aggregation bugs.

A small post-processing layer (remove backticks, pretty-print) helps productionize results.

It is important to include an “impossible question” test; it reveals whether the prompt truly blocks hallucination.

Bottom line.
V2 and V3 are production-grade: they consistently produce clean SQL or clean refusals. V1 is useful for quick prototyping but risks formatting drift and hallucinated fields. For grading and real use, prefer V3, with V2 as a solid fallback.